<a href="https://colab.research.google.com/github/baikunth2a/Lead-Score/blob/master/Website_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Selenium Webdriver

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

#Importing Files from Drive

In [ ]:
import pandas as pd
import sys
import os

from google.colab import drive
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/lead_gen')
%cd /content/gdrive/My\ Drive/lead_gen
excel_data = pd.read_excel('send_or_not.xlsx', sheet_name='13k')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/lead_gen


#Assigning Novelle label as initial ranking

In [ ]:
def ranks(x):
  rank = ""
  if "bad website" in x.lower():
    rank = 1
  elif "good website" in x.lower():
    rank = 2
  elif "website down" in x.lower():
    rank = 0
  return rank

In [ ]:
def clean(url):
  url = url.replace(' ','')
  if not 'http' in url:
      return str('http://'+url).lower()
  else:
    return url

In [ ]:
import numpy as np
data_i = excel_data[['website', 'Novelle Reason']].dropna()
data_i = data_i[data_i['Novelle Reason'].str.lower().str.contains('website')]
data_i['Novelle Reason'] = data_i['Novelle Reason'].apply(ranks)
data_i['website'] = data_i['website'].apply(clean)
data_i.sample(2)

,website,Novelle Reason
166,http://costabanderas.com.mx/,1
485,http://www.manzanares.com.mx,1


Checking number of good, bad and down websites

In [ ]:
data_i['Novelle Reason'].value_counts()

1    52
0     9
2     6
Name: Novelle Reason, dtype: int64

#Importing analyzer script

In [ ]:
%cd /content/gdrive/My\ Drive/lead_gen/Website_classifier
from analyzer import analyze
import browser

/content/gdrive/My Drive/lead_gen/Website_classifier


In [ ]:
data_i['website'].shape

(67,)

#Implementing multiprocessing to analyze websites

In [ ]:
from multiprocessing import Pool
def Browser():
    driver = browser.webdriver.Chrome(options=browser.chromeOptions, desired_capabilities=browser.caps)
    driver.set_page_load_timeout(30)
    return driver
 
results = []
def log_result(result):
    results.append(result)
    print(len(results))

def analysis(link):
    browser.driver = Browser()
    result = analyze(link)['pages'][0]
    browser.driver.close()
    return result

def multip():
    links = data_i['website']
    pool = Pool(processes=8)
    for link in links:
        pool.apply_async(analysis, args={link}, callback = log_result)
    pool.close()
    pool.join()
multip()

Converting list to dataframe

In [ ]:
results_d = pd.DataFrame(results)

In [ ]:
results_d.shape

(65, 17)

#Analyzing the extracted features

In [ ]:
results_d[results_d['best'] == True].describe()

,responsive,title_len,description_len,total_tag_len,h1_tags_len,word_count,image_len,Image_missing_alt_tag,og_title_l,link_count,anchor_missing_title_tag,anchor_text_contains_generic_text,total_time
count,6.0,6.000000,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000,6.000000,6.000000,6.000000,6.0,6.000000
mean,1.0,43.166667,21.500000,608.333333,1.666667,346.50000,22.500000,11.333333,0.333333,58.166667,56.666667,0.0,11.113806
std,0.0,21.207703,36.319416,368.053619,1.211060,221.18386,14.774979,18.337575,0.516398,55.829801,57.795040,0.0,4.717394
min,1.0,17.000000,0.000000,109.000000,0.000000,59.00000,3.000000,0.000000,0.000000,17.000000,1.000000,0.0,2.812551
25%,1.0,24.000000,0.000000,392.500000,1.000000,178.75000,17.500000,1.000000,0.000000,26.000000,31.250000,0.0,10.047590
50%,1.0,50.000000,4.000000,691.500000,1.500000,433.50000,19.000000,2.500000,0.000000,44.000000,44.000000,0.0,12.180181
75%,1.0,61.000000,23.750000,700.250000,2.750000,437.00000,27.250000,12.250000,0.750000,53.000000,53.000000,0.0,13.051093
max,1.0,62.000000,92.000000,1162.000000,3.000000,621.00000,47.000000,47.000000,1.000000,168.000000,168.000000,0.0,16.831799


Labeling best or not from the dataset

In [ ]:
results_d['best'] = results_d['url'].isin(data_i[data_i['Novelle Reason'] ==2]['website'])

#Printing best extracted **SEO features** of best websites (*According to Novelle*)

In [ ]:
results_d[results_d['best'] == True]

,url,status,responsive,title_len,description_len,total_tag_len,h1_tags_len,word_count,image_len,Image_missing_alt_tag,og_title_l,link_count,anchor_missing_title_tag,anchor_text_contains_generic_text,facebook_eng,total_time,best
1,http://www.aranzayasociados.com.mx,200,1,18,92,109,1,59,3,0,1,17,1,0,,2.812551,True
11,http://www.fypexherramental.com,200,1,17,8,1162,0,621,30,4,0,168,168,0,,9.477739,True
21,http://www.boltmedia.com.mx,200,1,42,29,294,2,95,47,47,1,23,30,0,,12.603219,True
24,http://www.cabocigars.net,200,1,62,0,688,3,437,19,1,0,53,53,0,,13.200384,True
27,https://www.cabocigars.net/,200,1,62,0,695,3,437,19,1,0,53,53,0,,11.757142,True
37,http://www.agrizar.com,200,1,58,0,702,1,430,17,15,0,35,35,0,,16.831799,True


#Calculating the score based on the best websites and SEO practices

In [ ]:
def calc_score(x):
  score = 0
  score = score + 10 if x['status'] == '200' else score
  score = score + 20 if x['responsive'] == 1 else score-5
  score = score + 3 if (x['title_len'] > 10 & x['title_len'] < 70)  else score
  score = score + 3 if (x['description_len'] > 1 & x['description_len'] < 255)  else score
  score = score + 10 if (x['total_tag_len'] > 200)  else score
  score = score + 5 if (x['h1_tags_len'] > 1 & x['h1_tags_len'] < 8)  else score - 2
  score = score + 5 if (x['word_count'] > 50 & x['word_count'] < 1000)  else score
  score = score + 10 if (x['image_len'] > 2 & x['image_len'] < 50)  else score-5
  score = score - 3 if (x['Image_missing_alt_tag']>2)  else score
  score = score + 8 if (x['og_title_l'])  else score
  score = score + 5 if (x['link_count'] > 10)  else score
  score = score + 5 if (x['facebook_eng'] != '')  else score
  return score

#Printing the websites having score greater than our threashold(63)

In [ ]:
def highlight_max(s):
  if s.score > 63:
    return ['background-color: yellow']*18
  else:
    return ['background-color: white']*18

In [ ]:
results_d['score'] = results_d.apply(calc_score, axis=1)
results_d.style.apply(highlight_max, axis=1)

,url,status,responsive,title_len,description_len,total_tag_len,h1_tags_len,word_count,image_len,Image_missing_alt_tag,og_title_l,link_count,anchor_missing_title_tag,anchor_text_contains_generic_text,facebook_eng,total_time,best,score
0,http://cronosmonterrey.com/,200,0,16,0,11,0,2,1,1,0,0,0,0,,3.063305,False,16
1,http://www.aranzayasociados.com.mx,200,1,18,92,109,1,59,3,0,1,17,1,0,,2.812551,True,64
2,http://caescontadores.com,200,0,49,0,29,0,27,1,1,0,0,0,0,,2.899691,False,21
3,http://servifront.com.mx/,200,0,24,0,169,0,130,9,9,0,15,15,0,,3.587351,False,23
4,http://www.viajesrangel.com.mx,200,0,26,0,76,10,22,2,1,0,0,1,0,,3.978696,False,13
5,http://www.avanceempresarial.com,200,0,18,0,278,0,126,25,20,0,77,45,0,,4.616708,False,33
6,https://factor02.com/,200,0,9,0,7,0,2,1,0,0,0,0,0,,1.079289,False,16
7,http://eseeni.com/,200,0,33,0,13,0,9,1,1,0,1,1,0,,1.260113,False,21
8,http://www.mactuarial.com.mx,200,0,12,0,27,0,14,0,0,0,0,0,0,,3.187795,False,6
9,http://www.energisol.com.mx,200,0,9,0,13,0,1,1,1,0,1,1,0,,2.066783,False,21


Total time for completion

In [ ]:
results_d['total_time'].sum()

394.7077453136444

#Further works


1.   We need more labled websites (Especially labled as good website)
2.   We need to know important features for the classification (Frontend Team)



